In [1]:
import random

In [2]:
import planetx_game.db_ops as db_ops
from planetx_game.board import SpaceObject
from planetx_game.board_type import sector_types
from planetx_game.session import *

In [7]:
class Session:
    def __init__(self, sessionID, code, board_length, gameID, first_rot, current_sector, current_action, 
                 game=None, players=None, theories=None, actions=None, history=None):
        self.sessionID = sessionID
        self.code = code
        self.board_length = board_length
        self.gameID = gameID
        self.first_rotation = first_rot
        self.current_sector = current_sector
        self.current_action = current_action
        self.game = game
        self.players = players
        self.theories = theories
        self.actions = actions
        self.history = history
        
    def __str__(self):
        return "Session " + self.code
    
    def __repr__(self):
        return "<Session " + self.code + ">"
    
    @staticmethod
    def _letter_to_int(letter):
        l = ord(letter) - 65
        if l < 8:
            return l
        elif l < 14:
            return l - 1
        else:
            return l - 2
        
    @staticmethod
    def _int_to_letter(i):
        if i < 8:
            return chr(i + 65)
        elif i < 13:
            return chr(i + 66)
        else:
            return chr(i + 67)
    
    @staticmethod
    def _code_to_int(code):
        i = Session._letter_to_int(code[0])
        code = code[1:]
        
        while len(code):
            i *= (24 * 8)
            d = int(code[0]) - 2
            l = Session._letter_to_int(code[1])
            
            i += d * 24
            i += l 
                        
            code = code[2:]
    
        return i
        
    @staticmethod
    def _int_to_code(i):
        code = Session._int_to_letter(i % 24)
        i = i // 24
        
        while i > 0:
            d = i % 8 + 2
            i = i // 8
            l = Session._int_to_letter(i % 24)
            i = i // 24
            code = l + str(d) + code
        
        if len(code) < 5:
            code = "A2" * ((5 - len(code))//2) + code
            
        return code
    
    @classmethod
    def create(cls, num_sectors):
        existing_codes = db_ops.get_session_codes()
        existing_ints = {Session._code_to_int(code) for code in existing_codes}
        available_ints = set(range(24*8*24*8*24)) - existing_ints
        
        chosen_int = random.choice(list(available_ints))
        chosen_code = Session._int_to_code(chosen_int)
        
        gid, _, game = db_ops.pick_game(num_sectors)
        
        session_id = db_ops.create_session(chosen_code, num_sectors, gid)
        
        return Session(session_id, chosen_code, num_sectors, gid, True, 0, Action(ActionType.START_GAME, None), game=game)
    
    @classmethod
    def find_by_code(cls, session_code):
        session_id, _, game_size, game_id, first_rot, sector, action_type, action_player = \
        db_ops.get_session_by_code(session_code)
        return Session(session_id, session_code, game_size, game_id, first_rot, \
                       sector, Action(ActionType[action_type], action_player))
    
    @classmethod
    def find_by_id(cls, session_id):
        _, session_code, game_size, game_id, first_rot, sector, action_type, action_player = \
        db_ops.get_session_by_id(session_id)
        return Session(session_id, session_code, game_size, game_id, first_rot, \
                       sector, Action(ActionType[action_type], action_player))
    
    def refresh_theories(self):
        self.theories = None
        return self.get_theories()
    
    def refresh_players(self):
        self.players = None
        return self.get_players()
    
    def refresh_actions(self):
        self.actions = None
        return self.get_actions()
    
    def refresh_history(self):
        self.history = None
        return self.get_history()
    
    def refresh_status(self):
        _, _, _, _, first_rot, current_sector, action_type, action_player = db_ops.get_session_by_id(self.sessionID)
        self.first_rotation = first_rot
        self.current_action = Action(ActionType[action_type], action_player)
        self.current_sector = current_sector
        
    def refresh(self):
        self.refresh_theories()
        self.refresh_players()
        self.refresh_actions()
        self.refresh_history()
        self.refresh_status()
        
    def get_theories(self):
        if self.theories is None:
            self.theories = db_ops.get_theories_for_session(self.sessionID)
        return self.theories
    
    def get_players(self):
        if self.players is None:
            self.players = db_ops.get_players_for_session(self.sessionID)
        return self.players
    
    def get_actions(self):
        if self.actions is None:
            self.actions = db_ops.get_current_actions_for_session(self.sessionID)
        return self.actions
    
    def get_history(self):
        if self.history is None:
            self.history = db_ops.get_previous_turns(self.sessionID)
        return self.history
    
    def get_game(self):
        if self.game is None:
            _, self.game = db_ops.get_game_by_id(self.gameID)
        return self.game
    
    def get_next_sector(self):
        players = sorted(self.get_players(), key=lambda p: p.sector)
        
        consecutive_pairs = zip(players, players[1:] + [players[0]])
        diffs = [(pair[1].sector, (pair[1].sector - pair[0].sector) % self.board_length) for pair in consecutive_pairs]
        start_sky = max(diffs, key=lambda d: d[1])[0]
        
        return start_sky
    
    def get_next_player_turn(self):
        next_sector = self.get_next_sector()
        
        next_player = min((player for player in self.get_players() if player.sector == next_sector), \
                          key=lambda p: p.arrival)
        
        return next_sector, next_player 
                          
    def game_json(self):
        return self.get_game().to_json()
        
    def state_json(self):
        return {
            "players": [player.to_json() for player in self.get_players()],
            "theories": [theory.to_json() for theory in self.get_theories()],
            "actions": [action.to_json() for action in self.get_actions()],
            "history": [turn.to_json() for turn in self.get_history()],
            "firstRotation": self.first_rotation,
            "currentSector": self.current_sector,
            "currentAction": self.current_action.to_json(),
            "sessionID": self.sessionID
        }

In [8]:
class SessionManager:  
    def join_session(self, session_code, name):
        session = Session.find_by_code(session_code)
        if session.current_action is None or session.current_action.action_type != ActionType.START_GAME:
            return False
        
        session_id, player_num, player_id = db_ops.new_player(session_code, name, False)
        # register subscriber
        return player_id
    
    def start_game(self, session_id, player_id):
        current_action = db_ops.get_current_action(player_id)
        if current_action is None or current_action.action_type != ActionType.START_GAME:
            return False
        
        db_ops.resolve_action(current_action.action_id, None)

        session = Session.find_by_id(session_id)
        
        self.randomize_order(session_id)
            
    def randomize_order(self, session_id):
        players = db_ops.get_players_for_session(session_id)
        random.shuffle(players)
        for i, player in enumerate(players):
            player.arrival = i + 1
            db_ops.move_player(player.player_id, player.sector, player.arrival)
        if len(players):
            action = Action(ActionType.PLAYER_TURN, players[0].player_id)
            db_ops.create_action(action.action_type, action.player_id)
            db_ops.set_current_action(session_id, action)
        #notify subscribers
    
    def next_action(self, session):
        sector_type = sector_types[session.board_length]
        
        current_sector = session.current_sector
        next_sector, next_player = session.get_next_player_turn()
        
        if next_sector < current_sector:
            next_sector += session.board_length
        
        sector, action = next_sector, Action(ActionType.PLAYER_TURN, next_player.player_id)
                
        theory_offset = (-current_sector - 1) % sector_type.theory_phase_interval
        next_theory = (current_sector + theory_offset) % session.board_length
        
        if session.current_action.action_type == ActionType.THEORY_PHASE or \
            session.current_action.action_type == ActionType.CONFERENCE_PHASE and current_sector == next_theory:
            next_theory += sector_type.theory_phase_interval
                
        if next_theory < sector:
            sector, action = next_theory, Action(ActionType.THEORY_PHASE, None)
        
        if session.first_rotation:
            try:
                if session.current_action.action_type != ActionType.CONFERENCE_PHASE:
                    next_conference = next(conference for conference in sector_type.conference_phases 
                                           if conference >= current_sector)
                else:
                    next_conference = next(conference for conference in sector_type.conference_phases 
                                           if conference > current_sector)

                if next_conference < sector:
                    sector, action = next_conference, Action(ActionType.CONFERENCE_PHASE, None)
            except StopIteration:
                pass
            
        sector = sector % session.board_length
        still_first_rot = sector >= current_sector and session.first_rotation
        
        db_ops.set_current_status(session.sessionID, action, sector, still_first_rot)
        
        if action.action_type == ActionType.PLAYER_TURN:
            db_ops.create_action(action.action_type, action.player_id)
        else:
            for player in session.get_players():
                db_ops.create_action(action.action_type, player.player_id)
                
        # notify subscribers?
    
    
    def last_action(self, session):
        db_ops.set_current_status(session.sessionID, ActionType.LAST_ACTION, None, session.first_rotation)
        
        for player in session.get_players():
            db_ops.create_action(ActionType.LAST_ACTION, player.player_id)
            
        #notify subscribers?
    
    def submit_theories(self, session_id, player_id, theories):
        current_action = db_ops.get_current_action(player_id)
        if current_action is None or (current_action.action_type != ActionType.THEORY_PHASE
                                      and current_action.action_type != ActionType.LAST_ACTION):
            return 0
        
        # TODO: Limit number of theories allowed on last action
                    
        session = Session.find_by_id(session_id)
        existing_theories = session.get_theories()
        num_objects = session.get_game().board.num_objects()
        
        tokens_left = num_objects.copy()
        for theory in existing_theories:
            if theory.player_id == player_id:
                tokens_left[theory.space_object] -= 1
                
        revealed_sectors = {theory.sector for theory in existing_theories if theory.revealed()}
        
        successful_theories = 0
        for theory in theories:
            if tokens_left[theory.space_object] and theory.sector not in revealed_sectors:
                db_ops.create_theory(session_id, player_id, theory.space_object.initial(), theory.sector)
                successful_theories += 1
                
        db_ops.resolve_action(current_action.action_id, None)
        actions = session.get_actions()
        if len(actions) == 0:
            self.next_action(session)
        #notify subscribers
        return successful_theories
    
    def read_conference(self, session_id, player_id):
        current_action = db_ops.get_current_action(player_id)
        if current_action is None or current_action.action_type != ActionType.CONFERENCE_PHASE:
            return False
        
        db_ops.resolve_action(current_action.action_id, None)
        session = Session.find_by_id(session_id)
        actions = session.get_actions()
        if len(actions) == 0:
            self.next_action(session)
        
        return True
    
    def advance_theories(self, session_id):
        db_ops.advance_theories(session_id)
        # notify subscribers
    
    def make_move(self, session_id, player_id, turn, sectors):
        current_action = db_ops.get_current_action(player_id)
        if current_action is None:
            return False
        if current_action.action_type == ActionType.LAST_ACTION and turn.turn_type != TurnType.LOCATE_PLANET_X:
            return False
        elif current_action.action_type != ActionType.PLAYER_TURN:
            return False
        
        if current_action.action_type != ActionType.LAST_ACTION:
            db_ops.advance_player(player_id, sectors)
        
        db_ops.resolve_action(current_action.action_id, turn)
        
        session = Session.find_by_id(session_id)
        self.next_action(session)
        # notify subscribers  
                
    def create_game(self, num_sectors, name):
        # register subscriber
        session = Session.create(num_sectors)
        session_id, player_num, player_id = db_ops.new_player(session.code, name, True)
        db_ops.set_current_action(session_id, Action(ActionType.START_GAME, player_id))
        session.refresh_status()
        return player_id, session

In [9]:
sm = SessionManager()

In [10]:
sm.create_game(12, "georgia")

(63, <Session V7X5T>)

In [11]:
s = Session.find_by_code("V7X5T")

In [31]:
s.refresh()
s.state_json()

{'players': [{'num': 1,
   'name': 'georgia',
   'sector': 8,
   'arrival': 1,
   'id': 63}],
 'theories': [],
 'actions': [{'actionType': 'PLAYER_TURN', 'playerID': 63}],
 'history': [{'playerID': 63,
   'type': 'RESEARCH',
   'selection': 'G',
   'sectors': [],
   'text': 'Research G ',
   'time': datetime.datetime(2021, 5, 2, 21, 31, 55)},
  {'playerID': 63,
   'type': 'RESEARCH',
   'selection': 'G',
   'sectors': [],
   'text': 'Research G ',
   'time': datetime.datetime(2021, 5, 2, 21, 32, 53)},
  {'playerID': 63,
   'type': 'RESEARCH',
   'selection': 'G',
   'sectors': [],
   'text': 'Research G ',
   'time': datetime.datetime(2021, 5, 2, 21, 33, 1)},
  {'playerID': 63,
   'type': 'RESEARCH',
   'selection': 'G',
   'sectors': [],
   'text': 'Research G ',
   'time': datetime.datetime(2021, 5, 2, 21, 33, 10)},
  {'playerID': 63,
   'type': 'RESEARCH',
   'selection': 'G',
   'sectors': [],
   'text': 'Research G ',
   'time': datetime.datetime(2021, 5, 2, 21, 33, 14)}],
 'first

In [148]:
sm.join_session("U5J5T", "rubie")

56

In [13]:
sm.start_game(48, 63)

In [27]:
sm.submit_theories(48, 63, [])

0

In [22]:
def m():
    sm.make_move(48, 63, Turn(TurnType.RESEARCH, "G", ()), 1)

In [32]:
import cProfile
cProfile.run("m()")

         4367 function calls in 0.540 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.540    0.540 <ipython-input-22-9f92bfdfb181>:1(m)
        3    0.000    0.000    0.052    0.017 <ipython-input-7-b46c7073df21>:139(get_players)
        1    0.000    0.000    0.052    0.052 <ipython-input-7-b46c7073df21>:159(get_next_sector)
        1    0.000    0.000    0.000    0.000 <ipython-input-7-b46c7073df21>:160(<lambda>)
        1    0.000    0.000    0.000    0.000 <ipython-input-7-b46c7073df21>:163(<listcomp>)
        1    0.000    0.000    0.000    0.000 <ipython-input-7-b46c7073df21>:164(<lambda>)
        1    0.000    0.000    0.052    0.052 <ipython-input-7-b46c7073df21>:168(get_next_player_turn)
        2    0.000    0.000    0.000    0.000 <ipython-input-7-b46c7073df21>:171(<genexpr>)
        1    0.000    0.000    0.000    0.000 <ipython-input-7-b46c7073df21>:172(<lambda>)
        1    0.

In [50]:
sm.read_conference(46, 62)

True

In [35]:
Turn(TurnType.SURVEY, (1,6)).code()

'S1,6'

In [22]:
help(Theory)

Help on class Theory in module planetx_game.session:

class Theory(builtins.object)
 |  Theory(space_object, sector, player_id, progress=0)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, space_object, sector, player_id, progress=0)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  __str__(self)
 |      Return str(self).
 |  
 |  revealed(self)
 |  
 |  to_json(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [33]:
app = FastAPI() 
endpoint = PubSubEndpoint()
endpoint.register_route(app, "/pubsub")
endpoint.publish(["my_event_topic"], data=["my", "data", 1])

NameError: name 'FastAPI' is not defined